# Homework: Hindcast skill assessment

Stine Fohrmann

![](plots/homework-stat-predict.png)


In [1]:
# Imports
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
from datetime import datetime
import scipy.stats as stats
from random import gauss, seed
from functions import *

In [2]:
data = read_from_file()

# Define area
lat_min    = 58
lat_max    = 63    # north‑south limits
lon_min    = 9
lon_max    = 13    # west‑east limits

# Select data for area around Oslo (in K)
data_subset = get_data_subset(data)

# Compute reference climatology for 1940-1969
climatology = compute_climatology(data)

# Index calculation (time series)
# Count HWD in JJA for each year
hwd_JJA_years, hwd_JJA = count_hwd(data_subset, climatology,
    start_month=6,
    end_month=8
)

# Persistence prediction
# Count HWD in MAM for each year
hwd_MAM_years, hwd_MAM = count_hwd(data_subset, climatology,
    start_month=3,
    end_month=5
)

# Climatology prediction
hwd_climatology_JJA_years, hwd_climatology_JJA = count_hwd(data_subset, climatology,
    start_year=1940,
    end_year=1969,
    start_month=6,
    end_month=8
)
hwd_avg = hwd_climatology_JJA.mean()
hwd_avg_array = np.full(len(hwd_JJA_years), hwd_avg)


**Anomaly correlation coefficient (ACC)**

$ACC = \frac{\sum_{k=1}^N (Y_k - \bar{Y}) (O_k - \bar{O})}{\sqrt{\sum_{k=1}^N (Y_k - \bar{Y})^2 \sum_{k=1}^N (O_k - \bar{O})^2}}$

**Root mean square error (RMSE)**

$RMSE = \sqrt{\frac{1}{N} \sum_{k=1}^N (Y_k - O_k)^2}$

## Persistence prediction skill

- ACC = 0.17 -> low correlation between prediction and observation

- RMSE = 5.65 -> higher than for climatology -> worse than climatology

In [3]:
# O_k: Observed number of summer HWDs
O_k = hwd_JJA
# Y_k: Predicted number of summer HWDs
Y_k = hwd_MAM

# O_avg: Observed average number of summer HWDs
O_avg = hwd_JJA.mean()
# Y_avg: Predicted average number of summer HWDs
Y_avg = hwd_MAM.mean()

print(f'O avg = {O_avg.item()}')
print(f'Y avg = {Y_avg.item()}')

acc = ACC(Ok=O_k, Oavg=O_avg, Yk=Y_k, Yavg=Y_avg)
rmse = RMSE(O_k, Y_k)

print(f'ACC = {acc.item()}')
print(f'RMSE = {rmse.item()}')

O avg = 2.8
Y avg = 3.0823529411764707
ACC = 0.16837385305625785
RMSE = 5.646446045184216


## Climatology prediction skill

- ACC = undef. (since no information about deviation from mean)

- RMSE = 3.682008900380832 -> lower than for persistence forecast -> better

In [4]:
# O_k: Observed number of summer HWDs
O_k = hwd_JJA
# Y_k: Predicted number of summer HWDs
Y_k = np.array(hwd_avg_array)

# O_avg: Observed average number of summer HWDs
O_avg = hwd_JJA.mean()
# Y_avg: Predicted average number of summer HWDs
Y_avg = hwd_avg

print(f'O avg = {O_avg.item()}')
# print(f'Yk = {Y_k}')
print(f'Y avg = {Y_avg.item()}')

rmse = RMSE(O_k, Y_k)

# print(f'ACC = {acc.item()}')
print(f'RMSE = {rmse.item()}')

O avg = 2.8
Y avg = 1.7666666666666666
RMSE = 3.682008900380832


## Analysis

Since the climatology's RMSE is lower than the one for the persistence prediction, climatology is on average a better predictor for the number of summer HWDs around Oslo than assuming spring persistence.

The low ACC of the persistence prediction confirms this, suggesting that assuming spring persistence adds very little predictive skill beyond a simple mean for this index.

This suggests that the amount of abnormally warm summer days around Oslo is not significantly correlated to the amount of abnormally warm days during the previous spring.

## Additional: Prediction based on previous years

In [5]:
# Compute full climatology of summer HWDs (for computing average of previous 5 years)
hwd_climatology_JJA_years_full, hwd_climatology_JJA_full = count_hwd(data_subset, climatology,
    start_year=1940,
    end_year=2024,
    start_month=6,
    end_month=8
)

In [6]:
# Number of years to use for prediction calculation
num_years = 5
# Calculate prediction based on previous years
previous_years_prediction = compute_previous_years_prediction(hwd_climatology_JJA_full, num_years)

O_k = hwd_JJA[num_years:-1]
Y_k = previous_years_prediction[num_years:-1]
O_avg = O_k.mean()
Y_avg = Y_k.mean()

acc = ACC(Ok=O_k, Oavg=O_avg, Yk=Y_k, Yavg=Y_avg)
rmse = RMSE(O_k, Y_k)

print(f'ACC = {acc.item()}')
print(f'RMSE = {rmse.item()}')
# O_k

ACC = 0.032808380016916494
RMSE = 3.8788618975484503


## Prediction based on May persistence

In [7]:
# Count HWD in May for each year
hwd_May_years, hwd_May = count_hwd(data_subset, climatology,
    start_month=5,
    end_month=5
)

In [9]:
O_k = hwd_JJA
Y_k = hwd_May
O_avg = O_k.mean()
Y_avg = Y_k.mean()

acc = ACC(Ok=O_k, Oavg=O_avg, Yk=Y_k, Yavg=Y_avg)
rmse = RMSE(O_k, Y_k)

print(f'ACC = {acc.item()}')
print(f'RMSE = {rmse.item()}')

ACC = 0.1655253466897079
RMSE = 4.632366946757539
